### Introduction
I'll build a recommender system using a modified version of Word2Vec, named Item2Vec ([1]), which consists in a collaborative filtering algorithm that aplies Word2Vec's Skip-Gram architecture on a non-natural language dataset. 

Word2Vec is a 1 hidden layer fully connected neural network that learns vector embeddings of words in a latent space. These embeddings are able to preserve word semantincs based on the their context (defined by a window size). Words that appear in similar contexts will tend to have similar vector representations. 
Word2Vec comes with 2 architecture options:
1. Continuous bag of words (CBOW): tries to predict a word based on its context.
2. Skip Gram (SG): tries to predict the context of a word that is given as an input. 

Since Item2Vec uses Skip-Gram, I'll only focus on its presentation.

The image below ([4]) shows the neural network architecture.
![skip_gram_image_placeholder](./../images/skip_gram_net_arch.png)

The input layer has the size of the vocabulary (10000 in the example above) and receives the one-hot vector representation of the word used for prediction.

The hidden layer size is equal to the dimension of learnt vector embeddings (300 in the example above) and represents a hyperparameter. The weight matrix will have 300 columns (the number of features) and 10000 rows (one for word in the vocabulary).

The output layer gives a probability distribution that will tell for each word in the vocabulary the probability of it being part of the input word's context.

There are some additional techniques that the authors of Word2Vec proposed in their second paper ([5]) in order to optimize the training of the neural network.

**Subsampling**\
It is used to decrease the number of training examples by removing the words that appear most frequently. Usually, these are stop words that do not have relevant semantic meaning, since they appear in all sorts of contexts. The probability of a word being removed is given by the following formula

![subsampling_image_placeholder](./../images/subsampling.png)

where f(w) represents the frequency of the given word in the vocabulary and t is a hyperparameter threshold usually chosen around 10<sup>-5</sup>.

**Negative sampling**
The training of this network will require a lot of time, since it implies the updating of the weight matrix which for a vocabulary of 10000 words and a hidden layer of 300 neurons will have **3 millions** weights. That's why a new technique called negative sampling has been introduced. This optimization implies modifing a very small sample of weights:
- the weight of the correct output neuron, so that it will result in a high probability of being part of the input word's context
- select a few (5-20) words from the vocabulary that will be treated as negative examples; Their weight will be updated so that their probability of being part of the input word's context will be decreased; For small vocabularies 5-20 negative samples are needed, whereas for larger vocabularies 2-5 samples are sufficient.
The probability of a word being picked as a negative example is given by the following formula

![negative_sampling_image_placeholder](./../images/negative_sampling.png)

where f(w) represents the frequency of the given word in the vocabulary. Therefore, most frequent words will have a higher probabily of being picked as negative examples.


**Item2Vec** aims to apply Word2Vec's Skip-Gram with negative sampling architecture on various sequences of objects belonging to non-linguistic datasets. Item2Vec can be applied to shopping cart items, movie reviews lists, lists of appreciated songs etc. in order to build a recommender system that is based on calculating the similarities of items that the user enjoys. Another characteristic of Item2Vec is that the window size is considered infinite because all the items in a given sequence of objects are considered to be part of the same context.

In this script I'll train this model on an online transactions dataset that contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail that sells unique all-occasion gifts ([6]).

The data has been explored and cleaned in separate scripts of this project. Check `data/initial_exploration` notebook and `data/make_dataset.py`.

In [1]:
import pandas as pd;
import numpy as np;
import logging

from random import shuffle
from sklearn.model_selection import KFold
from gensim.models import Word2Vec
logging.basicConfig(level=logging.INFO)


By grouping the purchased items based on their **InvoiceNo** we'll get a list of transactions. Each transaction will consist in a list of **StockCodes**.

We'll train a **Word2Vec** model that will try to learn the embeddings of the StockCodes in a latent space. Basically, we treat the purchased items (their *Stockcodes*) as "words" and the whole transaction (the sequence of items purchased together) as a "sentence". We can consider that items which are bought together are similar and belong to the same context.

Word2Vec hyperparameters were picked taking into account various values used in literature and based on personal experiments.

| Parameter name | Description | Value | Comments on the chosen values |
| --- | ----------- | ------- | ----- |
| size | Size of hidden layer (dimension of learned vectors) | 32 | Worked well for this use case. Multiple of 4 is chosen for greater performance (according to gensim documentation). |
| window | The size of window defining the context of the word | 999 | A big value, so all the products in a given transaction define the context used in learning |
| min_count | Minimum number of appearances for a word to be kept in the vocabulary | 4 | A product should appear at least 4 times in the whole corpus in order to be kept in the vocabulary |
| iter | Number of epochs | 30 | Tried: [10, 15, 20, 25, **30**, 35]. Highest score reached at 30 |
| sg | 1 for Skip-Gram strategy, 0 for CBOW | 1 | Skip-Gram technique works best for Item2Vec. According to Mikolov, Skip-Gram is a good solution for small datasets and is able to create a good represenation for rare words.|
| hs | 1 for hierarchical softmax, 0 and and negative > 0 for use of negative sampling | 0 | Chose 0 because applying negative-sampling gives better results | 
| sample | a threshold used for configuring the number of most popular words that are removed from the vocabulary | 5e-4 | Tried [e-4, **5e-4**, e-5].  |
| negative | how many random words are marked as negative examples | 30 | Tried: [5, 10, 20, **30**, 35]. Because the dataset is small, a higher value was needed.  |

After training the model, I evaluate it the following manner:\
For each transaction in the test set:
1. Extract the last item in the transaction. The model will try to predict this item based on the other items found in the transaction. If the extracted code is not part of the model's vocabulary, I'll keep extracting codes until the condition is met. I do so because the model will be unable to predict a 'word' that is not part of its vocabulary
2. Remove all the objects from the transaction that are not found in the vocabulary.
3. If the transaction is left empty after the first 2 steps, remove it from the test set.
4. Predict most similar K items to the ones from the transaction resulted on step 2. If the product extracted during the first step appears in the list of predictions, I'll increase the hit score.

The final score will be equal to the ratio between the total number of hits and the test set size. Three values have been considered for K: 10, 15, 20. Obviously, better scores are achived when K is larger. 

5-Folds Cross-validation has been used to compute the mean score of the model. The following scores have been obtained:
- for K = 10: 9.57%
- for K = 15: 11.91%
- for K = 20: 13.72%

In [5]:
class Transactions2Vec:
    def __init__(self):
        self.data = pd.read_csv('./data/data.csv', header=0)

        # Group entries by InvoiceNO 
        entryByInvoiceNo = self.data.groupby(["InvoiceNo"])
        # For each group extract the list of StockCodes and store them separately
        # These lists of StockCodes will be the "sentences" used in training the Word2Vec model
        self.transactions = [entryByInvoiceNo.get_group(gp)['StockCode'].tolist() for gp in entryByInvoiceNo.groups]

            
    def create_and_evaluate(self):
        self.scores = [[],[],[]]
        self.k_values = [10, 15, 20]
        
        # Use 5-Fold Cross-validation for model evaluation
        kf = KFold(n_splits=5, shuffle=True)
        iter = 1
        for train, test in kf.split(self.transactions):
            self.train_data = [self.transactions[x] for x in train]
            self.test_data = [self.transactions[x] for x in test]
            self.create_model(iter)
            self.evaluate_model()
            iter+=1

        # After getting a score for each of the 5 models, average them to get a mean score
        for i in range(len(self.k_values)):
            print("Mean score @{}= {}".format(self.k_values[i], np.mean(self.scores[i])))

    def create_model(self, iter):
        self.model = Word2Vec(self.train_data,
                              size=32,
                              window=999,
                              min_count=4,
                              workers=8,
                              iter=30,
                              sg=1,
                              hs=0,
                              sample=5e-4,
                              negative=30)
        
        # Save the model for each try
        model_name = "model_" + str(iter)
        self.model.save(model_name)

    def evaluate_model(self):
        test_size = float(len(self.test_data))
        hits = [0.0, 0.0, 0.0]
        
        for transaction in self.test_data:
            # If the transaction only contains a product, we skip it
            if len(transaction) < 2: 
                test_size -= 1.0
                continue
                
            # Extract the last bought item
            last_bought_product = transaction.pop() 
            # If the item is not found in the model's vocabulary, keep extracting other products
            while len(transaction) > 0 and last_bought_product not in self.model.wv.vocab:       
                last_bought_product = transaction.pop()
            # Create a list of all the remaining transaction's products that are part of the model's vocabulary
            products = [product for product in transaction if product in self.model.wv.vocab] 
            
            # If the list only contains a product, we skip this transaction
            if len(products) < 2:
                test_size -= 1.0
                continue
                
            # Predict top K products most similar to the ones extracted in the list above and check if the last_bought_item appears in the list of predictions
            for i in range(len(self.k_values)):
                prediction = self.model.wv.most_similar_cosmul(positive=products, topn=self.k_values[i])
                for predicted_product, score in prediction:
                    if predicted_product == last_bought_product:
                        hits[i] += 1.0

        # For each K value, compute the ratio between the number of correct predictions and the total test size.
        for i in range(len(hits)):
            score = hits[i] / test_size
            self.scores[i].append(score)
            print("Score @{}: {}".format(self.k_values[i], score))

In [6]:
model = Transactions2Vec()
model.create_and_evaluate()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
INFO:gensim.models.word2vec:collecting all words and their counts
INFO:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 241132 words, keeping 3677 word types
INFO:gensim.models.word2vec:collected 3901 word types from a corpus of 425288 raw words and 16582 sentences
INFO:gensim.models.word2vec:Loading a fresh vocabulary
INFO:gensim.models.word2vec:effective_min_count=4 retains 3485 unique words (89% of original 3901, drops 416)
INFO:gensim.models.word2vec:effective_min_count=4 leaves 424520 word corpus (99% of original 425288, drops 768)
INFO:gensim.models.word2vec:deleting the raw counts dictionary of 3901 items
INFO:gensim.models.word2vec:sample=0.000

Score @10: 0.1042147806004619
Score @15: 0.12817551963048499
Score @20: 0.14463048498845266


INFO:gensim.models.base_any2vec:training model with 8 workers on 3492 vocabulary and 32 features, using sg=1 hs=0 sample=0.0005 negative=30 window=999
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.92% examples, 3156 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 7.57% examples, 6230 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 10.26% examples, 6009 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 18.21% examples, 8814 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 25.17% examples, 11055 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 32.78% examples, 12005 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 46.94% examples, 14089 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 49.98% examples,

Score @10: 0.09371460928652321
Score @15: 0.12089467723669309
Score @20: 0.13901472253680633


INFO:gensim.models.base_any2vec:training model with 8 workers on 3493 vocabulary and 32 features, using sg=1 hs=0 sample=0.0005 negative=30 window=999
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.67% examples, 2470 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 10.63% examples, 6014 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 12.89% examples, 6439 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 23.35% examples, 9760 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 27.94% examples, 10533 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 30.97% examples, 10194 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 39.27% examples, 11520 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 49.53% examples

Score @10: 0.10014144271570014
Score @15: 0.12192362093352192
Score @20: 0.14172560113154173


INFO:gensim.models.base_any2vec:training model with 8 workers on 3487 vocabulary and 32 features, using sg=1 hs=0 sample=0.0005 negative=30 window=999
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.76% examples, 3043 words/s, in_qsize 16, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 7.51% examples, 4518 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 10.12% examples, 5157 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 19.69% examples, 8143 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 27.17% examples, 9629 words/s, in_qsize 16, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 34.99% examples, 11078 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 43.26% examples, 11796 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 48.56% examples, 

Score @10: 0.0893371757925072
Score @15: 0.11037463976945244
Score @20: 0.1256484149855908


INFO:gensim.models.base_any2vec:training model with 8 workers on 3477 vocabulary and 32 features, using sg=1 hs=0 sample=0.0005 negative=30 window=999
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 2.85% examples, 2428 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 7.65% examples, 5692 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 9.65% examples, 5203 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 14.76% examples, 6782 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 27.78% examples, 10145 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 30.91% examples, 10075 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 39.29% examples, 11190 words/s, in_qsize 15, out_qsize 0
INFO:gensim.models.base_any2vec:EPOCH 1 - PROGRESS: at 49.98% examples, 

Score @10: 0.09106480159862974
Score @15: 0.11390236939765915
Score @20: 0.13502711961176134
Mean score @10= 0.09569456199876444
Mean score @15= 0.11905416539356233
Mean score @20= 0.13720926865083055


## References
\[1\] Barkan, O., & Koenigstein, N. (2016). [ITEM2VEC: Neural item embedding for collaborative filtering](https://arxiv.org/abs/1603.04259). 2016 IEEE 26th International Workshop on Machine Learning for Signal Processing (MLSP), 1-6.

\[2\] Hugo Caselles-Dupre, Florian Lesaint, and Jimena Royo-Letelier. [Word2vec applied to recommendation: Hyperparameters matter](https://arxiv.org/abs/1804.04212). CoRR, abs/1804.04212, 2018.

\[3\] Makbule Gulcin Ozsoy. 2016. [From word embeddings to item recommendation](https://arxiv.org/abs/1601.01356). arXiv preprint arXiv:1601.01356 (2016).

\[4\] http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/

\[5\] Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. 2013b. [Distributed representations of words and phrases and their compositionality](https://arxiv.org/abs/1310.4546). In NIPS, pages 3111–3119.

\[6\] [Online Retail Data](https://archive.ics.uci.edu/ml/datasets/Online+Retail)